<a href="https://colab.research.google.com/github/olegsh60/RAG/blob/main/%D0%9F%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5_RAG_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_LLM_Qwen_2_5-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

#Ответы на вопросы с помощью LLM модели Qwen 7B + RAG

Устанавливаем общие библиотеки

In [1]:
# Библиотеки Langchain для взаимодействия с LLM
!pip install -U langchain langchain-community langchain-huggingface -q
# Библиотеки для вектороной DB FAISS и загрузки PDF
!pip install faiss-cpu pypdf -q
# Следующие три строки - установка библиотеки unsloth для использования квантованных LLM.
# Для Colab процесс имеет специфику по версиям библиотек. Если запускать не в Colab, то просто:
# !pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo -q
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/1

##Алгоритм создания RAG системы
1). Document Loading

In [3]:
# Загружаем файл "Вектореый поиск.pdf" для RAG, предварительно его нужно загрузить в Colab в раздел в левой пенели "Files"
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('/content/Векторный поиск.pdf')
documents = loader.load()
print("Число страниц: ", len(documents))

Число страниц:  4


2). Text Splitting — это процесс разбиения длинного текста на меньшие фрагменты (чаще всего — по абзацам, предложениям или символам), которые:
	•	легче обработать (в модели есть лимит токенов),
	•	проще индексировать в retrieval-системах,
	•	позволяют точнее связывать вопрос с контекстом.

In [4]:
# Определите Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

split_documents = splitter.split_documents(documents)

print("Число чанков: ", len(split_documents))


Число чанков:  11


3). Embedding это способ представления текста (или другого объекта) в виде числового вектора, который сохраняет смысл.

Простыми словами:

Это способ «перевести» текст в язык чисел, который понимает нейросеть.

Текст.    |Вектор (упрощённо).          |
----------|-----------------------------|
“кошка”.  |[0.12, -0.85, 0.33, …, 0.09  ]
“собака”. |[0.11, -0.83, 0.30, …, 0.12] |
“молекула”|[-0.45, 0.91, -0.12, …, 0.44]|

Вектора “кошка” и “собака” будут похожи, а “молекула” — далека.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

#  cuda - Use the GPU (if available)
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}
)

<ipython-input-5-7ef43698c20d>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

4). Storage. FAISS: Facebook AI Similarity Search

FAISS (читается «фэйс») — это библиотека от Facebook AI, предназначенная для поиска ближайших векторов (similarity search) в большом массиве данных.
Что делает FAISS?
	•	Быстро находит похожие элементы по вектору запроса.
	•	Поддерживает миллионы и миллиарды векторов.
	•	Работает на CPU и GPU.
	•	Используется в retrieval, поиске, рекомендательных системах, кластеризации.

In [6]:
from langchain.vectorstores import FAISS

db_embed = FAISS.from_documents(split_documents, hf_embeddings_model)
db_embed.save_local("faiss_db")

5). Retrieval.  
### Что такое Retrieval?

Retrieval (англ. извлечение) — это процесс поиска и возврата релевантной информации из большой базы данных или набора документов в ответ на запрос пользователя.

В отличие от генерации (когда модель сама «придумывает» ответ), retrieval ориентирован на нахождение уже существующего контента — точного, проверенного и хранимого где-то вне модели.

Как работает Retrieval-пайплайн?

1. Индексирование
	•	Все документы конвертируются в векторы (embeddings).
	•	Эти векторы сохраняются в векторную базу данных (например, FAISS, Qdrant, Weaviate).

2. Запрос
	•	Ввод пользователя также превращается в вектор.

3. Поиск ближайших документов
	•	Система находит наиболее похожие вектора-документы (на основе косинусного расстояния или L2).

4. Подача в LLM
	•	Извлечённые документы добавляются в prompt.
	•	Модель генерирует обоснованный ответ.

In [7]:
# Используем векторноое хранилище и его методов для получения документов
retriever = db_embed.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)


## Работа RAG системы


Импорт необходимых обьектов

In [8]:
from unsloth import FastLanguageModel
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import StrOutputParser

<ipython-input-8-22e63ddc0aef>:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Будем использовать Qwen 2.5 с 7B параметрами квантованную  в 4bit
https://huggingface.co/unsloth/Qwen2.5-7B-bnb-4bit


In [9]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Pipeline для формирования ответа LLM

In [10]:

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(".")]

pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=150, # обьем возвращаемых токенов
                repetition_penalty=1.2, # штрав за повторы в ответе немного увеличим от дефортной 1
                temperature=0.7, # Креативность уменьшим от дефортной 1 чтобы получать более точные ответы
                eos_token_id=terminators)

HF_model = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


Формируем шаблон для промпта

In [11]:
template = """
Отвечай на вопрос только используя следующий контекст:
{context}
Если ответа нет в контексте, то ответь: я не знаю!
Question: {question}
"""

# Определение шаблона промпта
prompt_template = PromptTemplate.from_template(template)

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

Запускаем RAG, очищаем ответ

In [12]:
query = "Что такое вектореый поиск?"
context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)

<ipython-input-12-b38f7361077b>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(query)



Отвечай на вопрос только используя следующий контекст:
интеллектуальных систем: от рекомендательных алгоритмов до поисков в базе знаний. 
Давайте разберёмся, что это такое, как это работает и почему это революция в мире 
поиска информации.
Что такое векторный поиск? 
В отличие от традиционного поиска по ключевым словам (keyword-based search), 
векторный поиск использует векторные представления (embeddings) текста. Каждый 
документ, предложение или даже слово преобразуется в вектор — набор чисел, 
отражающий его семантику.
📌  Простой пример:

Milvus, Qdrant, Weaviate), которая позволяет быстро искать ближайшие вектора по метрике 
расстояния (чаще всего — косинусное расстояние).
3. Поиск
Запрос пользователя тоже превращается в вектор. Затем алгоритм ищет вектора 
документов, ближайшие к вектору запроса. Результаты ранжируются по степени 
“похожести”.
Где используется векторный поиск? 
Векторный поиск — основа многих современных систем:
	 •	
 🔍  Поиск по базе знаний (RAG) — в LLM-система

Пример вопроса на тему не из документа

In [13]:
# Генерация ответа не по контексту
query = "Какое расстояние до луны?"

context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)


Отвечай на вопрос только используя следующий контекст:
текстами, LLM, знаниями или данными — игнорировать векторный поиск в 2025 году 
значит упустить целую эпоху возможностей.

Milvus, Qdrant, Weaviate), которая позволяет быстро искать ближайшие вектора по метрике 
расстояния (чаще всего — косинусное расстояние).
3. Поиск
Запрос пользователя тоже превращается в вектор. Затем алгоритм ищет вектора 
документов, ближайшие к вектору запроса. Результаты ранжируются по степени 
“похожести”.
Где используется векторный поиск? 
Векторный поиск — основа многих современных систем:
	 •	
 🔍  Поиск по базе знаний (RAG) — в LLM-системах сначала ищется контекст,

интеллектуальных систем: от рекомендательных алгоритмов до поисков в базе знаний. 
Давайте разберёмся, что это такое, как это работает и почему это революция в мире 
поиска информации.
Что такое векторный поиск? 
В отличие от традиционного поиска по ключевым словам (keyword-based search), 
векторный поиск использует векторные представления 

Пример запроса к чистой модели без использования RAG

In [14]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Что такое векторный поиск?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)

Вопрос: Что такое векторный поиск? Ответ: Векторный поиск — это метод поиска информации, который основан на использовании математических моделей и алгоритмов для анализа текста или других типов данных.


Пример вопроса к чистой модели на тему не из документа

In [15]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Какое расстяние до луны?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)

Вопрос: Какое расстяние до луны? Ответ: 384000 километров.
